# Imports and Dependencies

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from google.colab.patches import cv2_imshow
from tensorflow import keras
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
!unzip EyeData.zip

Archive:  EyeData.zip
   creating: EyeData/Train/
   creating: EyeData/Train/closed/
 extracting: EyeData/Train/closed/s0001_00197_0_0_0_0_0_01.png  
 extracting: EyeData/Train/closed/s0001_00439_0_0_0_0_1_01.png  
 extracting: EyeData/Train/closed/s0001_00458_0_0_0_0_0_01.png  
 extracting: EyeData/Train/closed/s0001_01603_0_1_0_0_0_01.png  
 extracting: EyeData/Train/closed/s0001_01662_0_1_0_0_1_01.png  
 extracting: EyeData/Train/closed/s0006_00299_0_1_0_0_0_01.png  
 extracting: EyeData/Train/closed/s0006_00476_0_1_0_0_0_01.png  
 extracting: EyeData/Train/closed/s0013_01851_0_1_0_0_0_01.png  
 extracting: EyeData/Train/closed/s0014_00010_0_0_0_0_0_01.png  
 extracting: EyeData/Train/closed/s0014_00011_0_0_0_0_0_01.png  
 extracting: EyeData/Train/closed/s0014_00091_0_0_0_0_0_01.png  
 extracting: EyeData/Train/closed/s0015_00144_0_0_0_0_1_01.png  
 extracting: EyeData/Train/closed/s0015_00155_0_0_0_0_1_01.png  
 extracting: EyeData/Train/closed/s0016_00294_1_0_0_0_1_01.png  
 extr

# Importing Training Data

In [ ]:
X=[]
y=[]

#Closed Eye
for i in tqdm(glob('/content/EyeData/Train/closed/*')):
  temp = np.array(Image.open(i).resize((64,64)))
  X.append(temp)
  y.append(1)

#Open Eye
for i in tqdm(glob('/content/EyeData/Train/open/*')):
  temp = np.array(Image.open(i).resize((64,64)))
  X.append(temp)
  y.append(0)

100%|██████████| 48/48 [00:00<00:00, 2899.75it/s]


In [ ]:
#Normalization
X = np.array(X)/255.0
y = np.array(y)

In [ ]:
#Train Test Split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(64,64)))
  hp_activation = hp.Choice('activation', values=['relu','tanh'])
  hp_learning_rate = hp.Choice('learning-rate', values=[1e-2, 1e-3, 1e-4])

  hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=128, step=16)
  hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=128, step=16)
  hp_layer_3 = hp.Int('layer_3', min_value=1, max_value=128, step=16)

  model.add(keras.layers.Dense(units=hp_layer_1, activation=hp_activation))
  model.add(keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
  model.add(keras.layers.Dense(2, activation='softmax'))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                                                loss=keras.losses.SparseCategoricalCrossentropy(),
                                                metrics=['accuracy'])
  return model

# Hyperparameter Optimization using Keras Tuner

In [ ]:
import keras_tuner as kt
tuner = kt.Hyperband( model_builder, objective='val_accuracy',
                     max_epochs=10, factor=8, directory='output',
                      project_name='drowsy_driver')

In [ ]:
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
tuner.search(x_train, y_train, epochs=3, validation_split=0.2, callbacks=[stop_early])

Trial 17 Complete [00h 00m 03s]
val_accuracy: 0.6153846383094788

Best val_accuracy So Far: 0.8461538553237915
Total elapsed time: 00h 01m 10s


Using the model with the parameters that give the greatest accuracy score.

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense (Dense)               (None, 33)                135201    
                                                                 
 dense_1 (Dense)             (None, 81)                2754      
                                                                 
 dense_2 (Dense)             (None, 2)                 164       
                                                                 
Total params: 138119 (539.53 KB)
Trainable params: 138119 (539.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, epochs=15, validation_split=0.2, initial_epoch=3)

Epoch 4/15
2/2 [==============================] - 2s 360ms/step - loss: 0.6733 - accuracy: 0.7255 - val_loss: 0.6693 - val_accuracy: 0.5385
Epoch 5/15
2/2 [==============================] - 0s 80ms/step - loss: 0.6711 - accuracy: 0.6078 - val_loss: 0.6670 - val_accuracy: 0.6154
Epoch 6/15
2/2 [==============================] - 0s 167ms/step - loss: 0.6651 - accuracy: 0.6667 - val_loss: 0.6598 - val_accuracy: 0.6923
Epoch 7/15
2/2 [==============================] - 0s 62ms/step - loss: 0.6644 - accuracy: 0.6863 - val_loss: 0.6524 - val_accuracy: 0.8462
Epoch 8/15
2/2 [==============================] - 0s 56ms/step - loss: 0.6576 - accuracy: 0.8235 - val_loss: 0.6492 - val_accuracy: 0.8462
Epoch 9/15
2/2 [==============================] - 0s 60ms/step - loss: 0.6546 - accuracy: 0.8824 - val_loss: 0.6449 - val_accuracy: 0.8462
Epoch 10/15
2/2 [==============================] - 0s 132ms/step - loss: 0.6502 - accuracy: 0.8235 - val_loss: 0.6424 - val_accuracy: 0.8462
Epoch 11/15
2/2 [======

In [ ]:
from keras.models import load_model

model.save('eye_open_closed_model.h5')  # creates a HDF5 file 'my_model.h5'

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
